### 正規化をしてテキストファイルに起こすノートブックです

In [1]:
import glob
import math

def split_list(l,n):
    for idx in range(0,len(l)-1,n):
        yield l[idx:idx + n]

def normalize_points(path,xmin,xmax,ymin,ymax):
    #LiDARからの生のデータを入力してね
    files = glob.glob(path)
    if len(files) == 0:
        print('No data available')
        return 
    with open(files[0],'r') as f:
        data = f.read().split('\n')
    normalize_x = []
    normalize_y = []
    for idx in range(1,len(data)-1):
        d_k = data[idx].replace('(','').replace(')',',').replace('[','').replace(']','')
        sp_ka = d_k.split(',')
        angl = [] #角度
        dist = [] #距離
        result = list(split_list(sp_ka,3))
        for idx in range(len(result)):
            angl.append(float(result[idx][1]))
            dist.append(float(result[idx][2]))

        points_x = []
        points_y = []
        for i in range(len(angl)):
            x = dist[i] * math.cos(math.radians(angl[i])) 
            y = math.sqrt(dist[i] ** 2 - x ** 2) 
            x = x / 10
            y = y / 10
            if(0 <= float(angl[i]) and float(angl[i]) <= 90):
                points_x.append(x)
                points_y.append(-y)
            elif(90 < float(angl[i]) and float(angl[i]) <= 180):
                points_x.append(x)
                points_y.append(-y)
            elif(180 < float(angl[i]) and float(angl[i]) <= 270):
                points_x.append(x)
                points_y.append(y)
            else:
                points_x.append(x)
                points_y.append(y)
        #ここで座標指定をしてしまうのがベスト？

        human_x = []
        human_y = []
        for point_x,point_y in zip(points_x,points_y):
            if xmin <= point_x and point_x <= xmax and ymin <= point_y and point_y <= ymax:
                human_x.append(point_x)
                human_y.append(point_y)
        
        max_points_x = max(human_x)
        min_points_x = min(human_x)
        max_points_y = max(human_y)
        min_points_y = min(human_y)

        bo_x = max_points_x - min_points_x
        bo_y = max_points_y - min_points_y
        n_x = []
        n_y = []
        for h_x in human_x:
            n_x.append((h_x - min_points_x) / bo_x)
        for h_y in human_y:
            n_y.append((h_y - min_points_y) / bo_y)
        normalize_x.append(n_x)
        normalize_y.append(n_y)
    return normalize_x,normalize_y

In [6]:
path = r'C:\Users\wtana\Desktop\LiDAR_cm\lidar_txt_files\tree_right_train.txt'
x,y = normalize_points(path,xmin=100,xmax=500,ymin=0,ymax=100)

In [7]:
import matplotlib.pyplot as plt
plt.plot(x[15],y[15],linestyle='',marker='*')
plt.grid('TRUE')
#plt.xlim(xmin,xmax)
#plt.ylim(ymin,ymax)

In [9]:
s = ''
for i in range(len(x)):
    s = ''
    with open('./normalize_txt/tree/'+str(i)+'.txt','a') as f:
        for x_,y_ in zip(x[i],y[i]):
            s = s + str(x_) + ' ' + str(y_) + ' 95\n'
        f.write(s)



In [9]:
s

'0.9120298838130259 1.0 95\n0.9000239267754437 0.9236224442617277 95\n0.8530375483182776 0.8432134516178998 95\n0.5897246564520376 0.7588806795274123 95\n0.39525592077157196 0.6827593365456596 95\n0.2545384555567279 0.6055014876481026 95\n0.03672297974033477 0.5315118036697246 95\n0.0 0.45826613209347034 95\n0.17286538039110108 0.3896190221509823 95\n0.41320668414052647 0.31622685727541205 95\n0.584577070657579 0.2394034603455049 95\n0.8497559604920741 0.1568736723968907 95\n0.8737120471348135 0.08029839250171875 95\n1.0 0.0 95\n'

In [8]:
from model import *
#from sampler import *
from self_made import *

loss = []
Accu = []

batch_size = 64
num_points = 64
num_labels = 1
def main_():
    pointnet = PointNet(num_points, num_labels)

    new_param = pointnet.state_dict()
    new_param['main.0.main.6.bias'] = torch.eye(3, 3).view(-1)
    new_param['main.3.main.6.bias'] = torch.eye(64, 64).view(-1)
    pointnet.load_state_dict(new_param)

    criterion = nn.BCELoss()
    optimizer = optim.Adam(pointnet.parameters(), lr=0.001)

    loss_list = []
    accuracy_list = []

    for iteration in range(200):

        pointnet.zero_grad()
        input_data, labels = data_sampler(path_1='./normalize_txt/takuya/150cm/*',path_2='./normalize_txt/tree/*')
        #input_data, labels = data_sampler(batch_size, num_points)

        output = pointnet(input_data)
        output = nn.Sigmoid()(output)

        error = criterion(output, labels)
        error.backward()

        optimizer.step()

        with torch.no_grad():
            output[output > 0.5] = 1
            output[output < 0.5] = 0
            accuracy = (output==labels).sum().item()/batch_size

        loss_list.append(error.item())
        accuracy_list.append(accuracy)
        loss.append(float(error.item()))
        Accu.append(float(accuracy))
        if iteration % 10 == 0:

            print('Iteration : {}   Loss : {}'.format(iteration, error.item()))
            print('Iteration : {}   Accuracy : {}'.format(iteration, accuracy))
            

In [9]:
main_()

Iteration : 0   Loss : 0.9023218154907227
Iteration : 0   Accuracy : 0.40625
Iteration : 10   Loss : 0.35822656750679016
Iteration : 10   Accuracy : 0.984375
Iteration : 20   Loss : 0.3260544240474701
Iteration : 20   Accuracy : 1.0
Iteration : 30   Loss : 0.31632497906684875
Iteration : 30   Accuracy : 1.0
Iteration : 40   Loss : 0.3135850727558136
Iteration : 40   Accuracy : 1.0
Iteration : 50   Loss : 0.31059491634368896
Iteration : 50   Accuracy : 1.0
Iteration : 60   Loss : 0.3036721348762512
Iteration : 60   Accuracy : 1.0
Iteration : 70   Loss : 0.29945793747901917
Iteration : 70   Accuracy : 1.0
Iteration : 80   Loss : 0.2960233986377716
Iteration : 80   Accuracy : 1.0
Iteration : 90   Loss : 0.29402703046798706
Iteration : 90   Accuracy : 1.0
Iteration : 100   Loss : 0.2919027805328369
Iteration : 100   Accuracy : 1.0
Iteration : 110   Loss : 0.292294979095459
Iteration : 110   Accuracy : 1.0
Iteration : 120   Loss : 0.29180315136909485
Iteration : 120   Accuracy : 1.0
Iterati

In [16]:
import matplotlib.pyplot as plt
a_len = len(Accu)
x_ = list(range(len(Accu)))
plt.plot(x_,Accu,marker='.',label='Accuracy')
plt.plot(x_,loss,marker='.',label='loss')
plt.legend()